<a href="https://colab.research.google.com/github/saxenam06/Image_search_UserText/blob/main/Store_Embeddings_LargeImageDrivingDataset_UsingBLIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount Drive


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Install Requirements, Load Libraries. Download & Define Dataset path, Create Spark Session


### Install requirements

In [2]:
!pip3 install pyspark
!pip install ipython-autotime
%load_ext autotime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=7c94c1732b346d61e68bf9f36860a2e4b3249bbfcca34c98b54e83d9a5014020
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.3 MB/s eta 0:00:00
time: 1.04 ms (started: 2023-09-07 10:35:42 +00:00)


### Load Libraries

In [3]:
import json
import os.path as osp
import os
import sys
from glob import glob
from PIL import Image
import pandas as pd
import numpy as np
import cv2
import random
import base64
from io import BytesIO
import io
import multiprocessing
import matplotlib.pyplot as plt
from IPython.display import HTML
from pyspark.sql.functions import udf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, pandas_udf, explode, PandasUDFType
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, StringType, ArrayType, BinaryType, DoubleType, FloatType

time: 1.03 s (started: 2023-09-07 10:35:42 +00:00)


### Download & Unzip data, Define DataSet Path
Download, Unzip data from https://once-for-auto-driving.github.io/index.html and Place it in your drive. Define Path below for Images and Annotations(MetaData) as per the drive location.

In [4]:
data_path = "/content/drive/MyDrive"
train_split_igs_folder = "ONCE_Training_split_cam_igs_unzip"
train_split_annotns_folder = "ONCE_Training_split_annotations_unzip"
train_split_igs_path = osp.join(data_path, train_split_igs_folder, 'data', '000076') # ,
train_split_annotns_path =osp.join(data_path, train_split_annotns_folder, 'data', '000076') # , '000076', , '000495'

time: 4.27 ms (started: 2023-09-07 10:35:43 +00:00)


### Create Spark Session

In [5]:
spark = SparkSession.builder.appName("Images_Embed").getOrCreate()
spark

time: 13.3 s (started: 2023-09-07 10:35:43 +00:00)


### Optimize configuration (As of now default!)

In [6]:
sc = spark.sparkContext
sc.getConf().getAll()

[('spark.app.name', 'Images_Embed'),
 ('spark.driver.port', '34319'),
 ('spark.driver.host', '7c0ab9d567d3'),
 ('spark.executor.id', 'driver'),
 ('spark.sql.warehouse.dir', 'file:/content/spark-warehouse'),
 ('spark.app.id', 'local-1694082950827'),
 ('spark.driver.extraJavaOptions',
  '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens

time: 121 ms (started: 2023-09-07 10:35:56 +00:00)


# Ingest Available images, Preprocess DataFrame, Load the Metadata, Integrate image dataframe with the Metadata

### Around 95k images and around 1000k JSON files consisiting Metadata and annotations for every image. Also find annotations from the JSON files for the available images and integrate to the image dataframe

### Load Images in binary format from drive

In [7]:
sample_img_dir = train_split_igs_path
spark = SparkSession.builder.appName("Images_Embed").getOrCreate()
image_df = spark.read.format("binaryFile") \
  .option("pathGlobFilter", "*.jpg") \
  .option("recursiveFileLookup", "true") \
  .load(sample_img_dir)

time: 41.5 s (started: 2023-09-07 10:35:56 +00:00)


In [8]:
image_df.rdd.getNumPartitions()

65

time: 1.56 s (started: 2023-09-07 10:36:38 +00:00)


### Preprocess: Add column for Frame_id (Optionally also for Seq_id and cam_id)

In [9]:
image_df.printSchema()

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)

time: 16.3 ms (started: 2023-09-07 10:36:39 +00:00)


In [10]:
def extract_cam_id(filepath):
    return os.path.basename(os.path.dirname(filepath))

def extract_seq_id(filepath):
    path_components = filepath.split('/')
    return path_components[-3]

def extract_frame_id(filepath):
    path_components = filepath.split('/')
    path_components = path_components[-1].split('.')
    return path_components[0]

# Register UDFs

extract_cam_id_udf = udf(extract_cam_id, StringType())
extract_seq_id_udf = udf(extract_seq_id, StringType())
extract_frame_id_udf = udf(extract_frame_id, StringType())
image_df = image_df.withColumn("Frame_id", extract_frame_id_udf(image_df["path"]))
image_df.printSchema()

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- Frame_id: string (nullable = true)

time: 107 ms (started: 2023-09-07 10:36:39 +00:00)


In [11]:
image_df = image_df.drop('length', 'modificationTime', 'path')
image_df.printSchema()

root
 |-- content: binary (nullable = true)
 |-- Frame_id: string (nullable = true)

time: 34.8 ms (started: 2023-09-07 10:36:40 +00:00)


### Define Schema and Load Metadata in JSON files

In [12]:
# Define the schema
schema = StructType([
    StructField("meta_info", StructType([
        StructField("weather", StringType(), nullable=True),
        StructField("period", StringType(), nullable=True),
    ])),
    StructField("frames", ArrayType(StructType([
        StructField("frame_id", StringType(), nullable=True),
    ])), nullable=True)
])

# Load JSON files and create a DataFrame
root_folder = train_split_annotns_path

json_df = spark.read.option("multiline", "true").json(root_folder + "/*", schema=schema)

# Explode the frames array to individual rows
json_df = json_df.select(
    col("meta_info.weather").alias("weather"),
    col("meta_info.period").alias("period"),
    explode(col("frames")).alias("frames"),
).selectExpr(
    "weather", "period",
    "frames.frame_id AS Frame_id",)

time: 5.9 s (started: 2023-09-07 10:36:40 +00:00)


### Check metadata and image schemas

In [13]:
json_df.printSchema()

root
 |-- weather: string (nullable = true)
 |-- period: string (nullable = true)
 |-- Frame_id: string (nullable = true)

time: 9.17 ms (started: 2023-09-07 10:36:45 +00:00)


In [14]:
image_df.printSchema()

root
 |-- content: binary (nullable = true)
 |-- Frame_id: string (nullable = true)

time: 10.2 ms (started: 2023-09-07 10:36:46 +00:00)


### Integrate Metadata with Image dataframe.
Add columns corresponding to weather, period and Frame_id from the JSON files to image_df using Frame_id as the joining key

In [15]:
selected_cols = ["Frame_id", "weather", "period"]
selected_json_df = json_df.select(*selected_cols)
image_df = image_df.join(selected_json_df, on="Frame_id", how="inner")

time: 148 ms (started: 2023-09-07 10:36:46 +00:00)


# Load reqd. Libraries for BLIP, Download BLIP model, Distribute model to every Spark executor node, Wrap model in Pandas UDF and Predict Embeddings using BLIP
Multi-node distributed inference of the model with PyTorch and Pandas UDF.
We use Pandas user-defined functions on Spark to implement a distributed, parallel approach for model inference.

### Load libraries

In [16]:
!pip install timm fairscale transformers
!pip install datasets torch
# install requirements
from pyspark import SparkFiles
import zipfile
import torch
import shutil
from torchvision.transforms.functional import InterpolationMode
from torchvision import transforms

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.3/266.3 kB 25.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 91.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 65.1 MB/s eta 0:00:00
  Created wheel for fairscale: filename=fairscale-0.4.13-py3-none-any.whl size=332106 sha256=90416e1d5d22f13c0aba55a832023dd495ffeec645e71a48771edeab23ed111c
  Stored in directory: /root/.cache/pip/wheels/78/a4/c0/fb0a7ef03cff161611c3fa40c6cf898f76e58ec421b88e8cb3
Successfully built fairscale
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6

### Download BLIP repo

In [17]:
if 'google.colab' in sys.modules:
    print('Running in Colab.')
    !git clone https://github.com/salesforce/BLIP
    %cd

Running in Colab.
Cloning into 'BLIP'...
remote: Enumerating objects: 277, done.
remote: Counting objects: 100% (165/165), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 277 (delta 137), reused 136 (delta 135), pack-reused 112
Receiving objects: 100% (277/277), 7.03 MiB | 14.88 MiB/s, done.
Resolving deltas: 100% (153/153), done.
/root
time: 2.32 s (started: 2023-09-07 10:37:35 +00:00)


### Distribute repo to every Spark executor node

In [18]:
sc = spark.sparkContext

# Distribute the ZIP archives
shutil.make_archive("/content/BLIP/models_zip", 'zip', "/content/BLIP", "models")
shutil.make_archive("/content/BLIP/configs_zip", 'zip', "/content/BLIP", "configs")

# Add the ZIP files to Spark
models_zip_path = '/content/BLIP/models_zip.zip'
configs_zip_path = '/content/BLIP/configs_zip.zip'

sc = spark.sparkContext
sc.addPyFile(models_zip_path)
sc.addPyFile(configs_zip_path)

sys.path.append('/content/BLIP/models')
sys.path.append('/content/BLIP/configs')

time: 112 ms (started: 2023-09-07 10:37:37 +00:00)


### Wrap model in Pandas UDF and Do distributed inference for Creating Embeddings using BLIP- model_base_retrieval_coco

Let Spark optimize the execution instead of writing imperative RDD code.

Define a Pandas UDF for the inference task and wrap the model.

In [19]:
from models.blip_itm import blip_itm

def initialize_blip_itm_model():
    model_url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_base_retrieval_coco.pth'
    model = blip_itm(pretrained=model_url, image_size=384, vit='base')
    model.eval()
    model = model.to(device='cpu')
    return model

# Define the schema for the output DataFrame
schema = StructType([
    StructField('Embeddings', ArrayType(FloatType()))
])

# Define a Pandas UDF to calculate ITM and ITC scores
@pandas_udf(schema)
def calculate_embeds(content_series: pd.Series) -> pd.DataFrame:

    models_zip_path_on_workers = SparkFiles.get('models_zip.zip')
    configs_zip_path_on_workers = SparkFiles.get('configs_zip.zip')

    models_subdirectory_path = '/content/'
    configs_subdirectory_path = '/content/'

    print(f"Extracting models to {models_subdirectory_path}")
    with zipfile.ZipFile(models_zip_path_on_workers, 'r') as zip_ref:
      zip_ref.extractall(models_subdirectory_path)

    print(f"Extracting models to {configs_subdirectory_path}")
    with zipfile.ZipFile(configs_zip_path_on_workers, 'r') as zip_ref:
      zip_ref.extractall(configs_subdirectory_path)

    med_config_path = '/content/configs/med_config.json'
    with open(med_config_path, 'r', encoding='utf-8') as reader:
      med_config_content = reader.read()
    print(f"med_config_path: {med_config_path}")

    model = initialize_blip_itm_model()
    model.eval()
    embeddings_list = []

    for content in content_series:
        image = Image.open(io.BytesIO(content))
        transform = transforms.Compose([
            transforms.Resize((384, 384), interpolation=InterpolationMode.BICUBIC),
            transforms.ToTensor(),
            transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
        ])
        image_tensor = transform(image).unsqueeze(0).to(device='cpu')  # Convert image to tensor

        # Perform your BLIP model calculations here
        image_embeds = model.visual_encoder(image_tensor)
        # embeddings_list.append(image_embeds.squeeze().detach().cpu().numpy())  # Append the tensor as isW

        # Flatten the image_embeds tensor and append it to the list
        flattened_embeds = image_embeds.squeeze().view(-1).detach().cpu().numpy()
        embeddings_list.append(flattened_embeds)

    return pd.DataFrame({'Embeddings': embeddings_list})

time: 6.7 s (started: 2023-09-07 10:37:37 +00:00)


In [20]:
image_df_embeds = image_df.withColumn("Embeddings", calculate_embeds(col("content")))
image_df_embeds.show(1)

+-------------+--------------------+-------+-------+--------------------+
|     Frame_id|             content|weather| period|          Embeddings|
+-------------+--------------------+-------+-------+--------------------+
|1616343577200|[FF D8 FF E0 00 1...|  sunny|morning|{[-0.39624017, 0....|
+-------------+--------------------+-------+-------+--------------------+
only showing top 1 row

time: 3min 3s (started: 2023-09-07 10:37:44 +00:00)


### Cache the Spark DataFrame and Write the Embeddings in Parquet format

In [ ]:
# Select and write the desired columns
image_df_embeds.cache()
image_df_embeds.select("Frame_id", "Embeddings") \
        .write.mode("overwrite") \
        .parquet("Frame_Embeds")

### Store the Embeddings to desired folder

In [ ]:
result_path = '/content/Frame_Embeds'
destination_directory = '/content/drive/MyDrive/Frame_Embeds_000076'
shutil.rmtree(destination_directory, ignore_errors=True)
shutil.copytree(result_path, destination_directory)
